In [2]:
import json
import pandas as pd
import re

In [3]:
with open('uniprot_json.json') as uniprot_json:
    uniprot_data = json.load(uniprot_json)

In [4]:
print(uniprot_data.keys())

dict_keys(['results'])


There is only one key and that is results

In [5]:
protein_access = uniprot_data['results']

In [6]:
print(protein_access[0])

{'entryType': 'UniProtKB reviewed (Swiss-Prot)', 'primaryAccession': 'A0A075TMP8', 'uniProtkbId': 'PATI_PENEN', 'entryAudit': {'firstPublicDate': '2018-12-05', 'lastAnnotationUpdateDate': '2022-12-14', 'lastSequenceUpdateDate': '2014-10-29', 'entryVersion': 31, 'sequenceVersion': 1}, 'annotationScore': 5.0, 'organism': {'scientificName': 'Penicillium expansum', 'commonName': 'Blue mold rot fungus', 'taxonId': 27334, 'lineage': ['Eukaryota', 'Fungi', 'Dikarya', 'Ascomycota', 'Pezizomycotina', 'Eurotiomycetes', 'Eurotiomycetidae', 'Eurotiales', 'Aspergillaceae', 'Penicillium']}, 'proteinExistence': '1: Evidence at protein level', 'proteinDescription': {'recommendedName': {'fullName': {'evidences': [{'evidenceCode': 'ECO:0000303', 'source': 'PubMed', 'id': '25120234'}], 'value': 'Cytochrome P450 monooxygenase patI'}, 'ecNumbers': [{'evidences': [{'evidenceCode': 'ECO:0000269', 'source': 'PubMed', 'id': '30680886'}], 'value': '1.-.-.-'}]}, 'alternativeNames': [{'fullName': {'evidences': [{

In [7]:
protein_access[0].keys()

dict_keys(['entryType', 'primaryAccession', 'uniProtkbId', 'entryAudit', 'annotationScore', 'organism', 'proteinExistence', 'proteinDescription', 'genes', 'comments', 'features', 'keywords', 'references', 'uniProtKBCrossReferences', 'sequence', 'extraAttributes'])

I need a for cycle which will go through all proteins. There I'm going to look at primaryAccession where the Uniprot ID of the protein is. I'm also going to look at key 'comments' where we can find all the information abou the reaction.

In [8]:
print(len(protein_access))

1726


We have 1726 reviewed entries in this json file

In [15]:
data = []

for i in range(len(protein_access)):
    id = protein_access[i]['primaryAccession']
    ptDescription = protein_access[i]['proteinDescription']
    name = ptDescription['recommendedName']['fullName']['value']
    comments = protein_access[i]['comments']
    reaction_found = False
    cofactor_found = False

    for j in range(len(comments)):
        if comments[j]['commentType'] == 'COFACTOR':
            cofactors = comments[j]['cofactors']
            cofactor = cofactors[0]['name']
            #print(cofactor)
        if comments[j]['commentType'] == "CATALYTIC ACTIVITY":
            reaction = comments[j]['reaction']['name']
            reactionSplit = reaction.split(' = ')
            substrates = reactionSplit[0].strip()
            products = reactionSplit[1].strip()
            
            substratesSplit = substrates.split(' + ')
            productsSplit = products.split(' + ')
            participants = substratesSplit + productsSplit
            
            reducing_agent = '' 
            new_substrates = []
            new_products = []
            for sub in productsSplit:
                if sub.strip() not in ['H(+)', 'H2O', 'AH2', '2 H(+)', '2 H2O','3 H(+)', '3 H2O', '4 H(+)', '4 H2O', '6 H2O', 'oxidized [NADPH--hemoprotein reductase]', 
                           'oxidized [2Fe-2S]-[ferredoxin]', 'oxidized [adrenodoxin]', '2 oxidized [NADPH--hemoprotein reductase]', '6 oxidized [NADPH--hemoprotein reductase]', 
                           '2 oxidized [2Fe-2S]-[ferredoxin]', '2 oxidized [adrenodoxin]', '6 oxidized [2Fe-2S]-[ferredoxin]', '6 oxidized [adrenodoxin]',
                           'A', 'NAD(+)', 'NADP(+)', 'NADPH', 'NADH',  '2 NADP(+)', '2 NADPH', '2 NADH', 'CO2', '2 CO2', '3 CO2',  '3 oxidized [NADPH--hemoprotein reductase]', 
                           '3 oxidized [2Fe-2S]-[ferredoxin]', '3 oxidized [adrenodoxin]', '4 oxidized [NADPH--hemoprotein reductase]', 
                           '4 oxidized [2Fe-2S]-[ferredoxin]', '4 oxidized [adrenodoxin]', 'oxidized [NADH--hemoprotein reductase]', '2 oxidized [flavodoxin]',  'oxidized 2[4Fe-4S]-[ferredoxin]']:
                    new_products.append(sub.strip())
            
            for sub in substratesSplit:
                if 'reduced [' in sub:
                    reducing_agent = sub.strip()[0:9] + sub.strip()[9:-1] + sub.strip()[-1:]
                elif '2 reduced ferredoxin [iron-sulfur] cluster' in sub:
                    reducing_agent =  '2 reduced ferredoxin [iron-sulfur] cluster'
                elif 'reduced 2[4Fe-4S]-[ferredoxin]' in sub:
                    reducing_agent =  'reduced 2[4Fe-4S]-[ferredoxin]'
                elif sub in ['NADPH', 'NADP(+)', 'NAD(+)', 'NADH', '2 NADPH', '3 NADPH']:
                    reducing_agent= sub.strip()
                elif sub not in ['O2', 'H(+)', 'AH2', 'H2O', 'H2O2', 'A', '2 O2', '2 H(+)', '2 AH2', '2 H2O', '2 H2O2', 
                                 '2 A', '3 O2', '3 H(+)', '3 AH2', '3 H2O', '3 H2O2', '3 A', '4 H(+)', '5 H(+)', '6 H(+)', '4 O2']:
                    new_substrates.append(sub.strip())

                    
            crossRef = comments[j]['reaction'].get('reactionCrossReferences', [])
            
            rheaID = ''
            chebis = []
            participant_chebis = {}
            for ref in crossRef:
                if ref['database'] == 'Rhea':
                    rheaID = ref['id']
                    break
                     
            
            data.append({'protein': id, 'name': name, 'reaction': reaction, 'rheaID': rheaID, 'substrates': new_substrates, 
                         'Substrate CHEBI ID': None, 'Substrate SMILES': None, 'reducing_agent': reducing_agent, 'cofactor': cofactor,
                         'products': new_products, 'Product CHEBI ID': None, 'Product SMILES': None, 
                         'participants': participants})
            reaction_found = True
            cofactor_found = True


    # before adding 2929 records in dataframe, after adding this 3781 records - so we have 2929 fully annotated reactions from reviewed 'PF00067'
    if not reaction_found and not cofactor_found:
        data.append({'protein': id, 'name': name, 'reaction': None, 'rheaID': None, 'reducing_agent': None,'cofactor': None, 'substrates': None, 
                         'products': None,'participants': None})

df = pd.DataFrame(data)

df.to_csv('reactionsfromjson.csv', index=False)
#df.to_excel('reactionFromJSONexcel.xlsx', index=False)

Now we need to add ChEBI IDs & SMILES of the substrates and the products to the dataframe from reaction_data.csv file.

In [17]:
reaction_data = pd.read_csv("reaction_data.csv")
reaction_data.head(20)

/tmp/ipykernel_31446/4129889877.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  reaction_data = pd.read_csv("reaction_data.csv")


,protein,reaction,reaction_side_order,chebi,smiles
0,Q7Z1V1,54028,1,CHEBI_138029,C12C([C@]3(C(C(CC3)*)(C)CC1)C)CCC4C2(CCCC4)C
1,Q7Z1V1,54028,1,CHEBI_15379,O=O
2,Q7Z1V1,54028,1,CHEBI_57618,Cc1cc2Nc3c([nH]c(=O)[nH]c3=O)N(C[C@H](O)[C@H](...
3,Q7Z1V1,54028,2,CHEBI_138031,C12C(C=3C(C(CC3)*)(C)CC1)CCC4C2(CCCC4)C
4,Q7Z1V1,54028,2,CHEBI_15740,[H]C([O-])=O
5,Q7Z1V1,54028,2,CHEBI_15378,[H+]
6,Q7Z1V1,54028,2,CHEBI_15377,[H]O[H]
7,Q7Z1V1,54028,2,CHEBI_58210,C12=NC([N-]C(C1=NC=3C(N2C[C@@H]([C@@H]([C@@H](...
8,Q9LJK2,12897,1,CHEBI_37569,CC(\C=C\[C@@]1(O)C(C)=CC(=O)CC1(C)C)=C\C([O-])=O
9,Q9LJK2,12897,1,CHEBI_15379,O=O


In [18]:
#getting rid of 'RHEA:' in my df to simplify the search of ChEBI IDs and SMILES in reaction_data.csv file
df['rheaID'] = df['rheaID'].str.replace("RHEA:","")

,protein,name,reaction,rheaID,substrates,Substrate CHEBI ID,Substrate SMILES,reducing_agent,cofactor,products,Product CHEBI ID,Product SMILES,participants
0,A0A075TMP8,Cytochrome P450 monooxygenase patI,3-hydroxybenzyl alcohol + O2 + reduced [NADPH-...,62212,[3-hydroxybenzyl alcohol],NaN,NaN,reduced [NADPH--hemoprotein reductase],heme,[gentisyl alcohol],NaN,NaN,"[3-hydroxybenzyl alcohol, O2, reduced [NADPH--..."
1,A0A075TRL5,Cytochrome P450 monooxygenase patH,3-methylphenol + O2 + reduced [NADPH--hemoprot...,62208,[3-methylphenol],NaN,NaN,reduced [NADPH--hemoprotein reductase],heme,[3-hydroxybenzyl alcohol],NaN,NaN,"[3-methylphenol, O2, reduced [NADPH--hemoprote..."
2,A0A087X1C5,Putative cytochrome P450 2D7,an organic molecule + O2 + reduced [NADPH--hem...,17149,[an organic molecule],NaN,NaN,reduced [NADPH--hemoprotein reductase],heme,[an alcohol],NaN,NaN,"[an organic molecule, O2, reduced [NADPH--hemo..."
3,A0A0C5Q4Y6,Ferruginol synthase 1,"abieta-8,11,13-triene + O2 + reduced [NADPH--h...",48080,"[abieta-8,11,13-triene]",NaN,NaN,reduced [NADPH--hemoprotein reductase],heme,[ferruginol],NaN,NaN,"[abieta-8,11,13-triene, O2, reduced [NADPH--he..."
4,A0A0C5Q4Y6,Ferruginol synthase 1,ferruginol + O2 + reduced [NADPH--hemoprotein ...,55428,[ferruginol],NaN,NaN,reduced [NADPH--hemoprotein reductase],heme,[11-hydroxyferruginol],NaN,NaN,"[ferruginol, O2, reduced [NADPH--hemoprotein r..."
5,A0A0C5Q4Y6,Ferruginol synthase 1,miltiradiene + 2 O2 + 2 reduced [NADPH--hemopr...,66796,[miltiradiene],NaN,NaN,2 reduced [NADPH--hemoprotein reductase],heme,[11-oxomiltiradiene],NaN,NaN,"[miltiradiene, 2 O2, 2 reduced [NADPH--hemopro..."
6,A0A0C5QRZ2,Ferruginol synthase,"abieta-8,11,13-triene + O2 + reduced [NADPH--h...",48080,"[abieta-8,11,13-triene]",NaN,NaN,reduced [NADPH--hemoprotein reductase],heme,[ferruginol],NaN,NaN,"[abieta-8,11,13-triene, O2, reduced [NADPH--he..."
7,A0A0C5QRZ2,Ferruginol synthase,ferruginol + O2 + reduced [NADPH--hemoprotein ...,55428,[ferruginol],NaN,NaN,reduced [NADPH--hemoprotein reductase],heme,[11-hydroxyferruginol],NaN,NaN,"[ferruginol, O2, reduced [NADPH--hemoprotein r..."
8,A0A0C5QRZ2,Ferruginol synthase,miltiradiene + 2 O2 + 2 reduced [NADPH--hemopr...,66796,[miltiradiene],NaN,NaN,2 reduced [NADPH--hemoprotein reductase],heme,[11-oxomiltiradiene],NaN,NaN,"[miltiradiene, 2 O2, 2 reduced [NADPH--hemopro..."
9,A0A1D6HSP4,Dimethylnonatriene synthase,"(6E,10E)-geranyllinalool + O2 + reduced [NADPH...",13545,"[(6E,10E)-geranyllinalool]",NaN,NaN,reduced [NADPH--hemoprotein reductase],heme,"[(3E,7E)-4,8,12-trimethyltrideca 1,3,7,11-tetr...",NaN,NaN,"[(6E,10E)-geranyllinalool, O2, reduced [NADPH-..."


Create a dictionary from the reaction_data dataframe, where tuple protein, reaction will be the key and value will be the array of chebis and smiles

In [19]:
# creating dictionary from reaction_data where key = (protein, reaction), value = list of remaining columns
protein_reaction = reaction_data.groupby(['protein', 'reaction'])
reaction_dict = {(k[0], k[1]): [list(row[3:]) for _, row in g.iterrows()] for k, g in protein_reaction}

In [20]:
# just an example of the values for key ("Q7Z1V1", "54028")
print(reaction_dict[("Q7Z1V1","54028")])

[['CHEBI_138029', 'C12C([C@]3(C(C(CC3)*)(C)CC1)C)CCC4C2(CCCC4)C'], ['CHEBI_15379', 'O=O'], ['CHEBI_57618', 'Cc1cc2Nc3c([nH]c(=O)[nH]c3=O)N(C[C@H](O)[C@H](O)[C@H](O)COP([O-])([O-])=O)c2cc1C'], ['CHEBI_138031', 'C12C(C=3C(C(CC3)*)(C)CC1)CCC4C2(CCCC4)C'], ['CHEBI_15740', '[H]C([O-])=O'], ['CHEBI_15378', '[H+]'], ['CHEBI_15377', '[H]O[H]'], ['CHEBI_58210', 'C12=NC([N-]C(C1=NC=3C(N2C[C@@H]([C@@H]([C@@H](COP(=O)([O-])[O-])O)O)O)=CC(=C(C3)C)C)=O)=O']]


In [21]:
# iterate through each row of df - find the tuple (id,rheaID) in the reaction_dict
# also get the index of each substrate and the index of each product in the participants list 
for index, row in df.iterrows():
    substrate_indexes = []
    product_indexes = []
    if row['substrates'] and row['products']:
        for subs in row['substrates']:
            if subs in row['participants']:
                substrate_indexes.append(row['participants'].index(subs))
        for prod in row['products']:
            if prod in row['participants']:
                product_indexes.append(row['participants'].index(prod))
    idXrheaID = (row['protein'], row['rheaID'])
    #print(idXrheaID,substrate_indexes, product_indexes)

    # now we need to find ChEBI ID & SMILES in the reaction_dict based on the tuple & indexes we got here

    # retrieve the list of remaining columns for the idXrheaID from the reaction_dict
    if idXrheaID in reaction_dict:
        all_vals = reaction_dict[idXrheaID]
        maxi = len(all_vals) - 1

        # iterate through the substrate_indexes and product_indexes lists to retrieve the corresponding values
        substrates = [all_vals[i] for i in substrate_indexes if i <= maxi]
        products = [all_vals[i] for i in product_indexes if i <= maxi]

        substrate_ids = [s[0] for s in substrates]
        substrate_smiles = [s[1] for s in substrates]
        product_ids = [p[0] for p in products]
        product_smiles = [p[1] for p in products]

        df.loc[index, 'Substrate CHEBI ID'] = ";".join(substrate_ids)
        df.loc[index, 'Substrate SMILES'] = ";".join(substrate_smiles)
        df.loc[index, 'Product CHEBI ID'] = ";".join(product_ids)
        df.loc[index, 'Product SMILES'] = ";".join(product_smiles)
    
    

df['products'] = df['products'].apply(lambda x: ';'.join(x) if isinstance(x,list) else x)
df['substrates'] = df['substrates'].apply(lambda x: ';'.join(x) if isinstance(x,list) else x)

df.head(20)

        

,protein,name,reaction,rheaID,substrates,Substrate CHEBI ID,Substrate SMILES,reducing_agent,cofactor,products,Product CHEBI ID,Product SMILES,participants
0,A0A075TMP8,Cytochrome P450 monooxygenase patI,3-hydroxybenzyl alcohol + O2 + reduced [NADPH-...,62212,3-hydroxybenzyl alcohol,CHEBI_17069,OCc1cccc(O)c1,reduced [NADPH--hemoprotein reductase],heme,gentisyl alcohol,CHEBI_5325,C=1(C=C(C(=CC1)O)CO)O,"[3-hydroxybenzyl alcohol, O2, reduced [NADPH--..."
1,A0A075TRL5,Cytochrome P450 monooxygenase patH,3-methylphenol + O2 + reduced [NADPH--hemoprot...,62208,3-methylphenol,CHEBI_17231,Cc1cccc(O)c1,reduced [NADPH--hemoprotein reductase],heme,3-hydroxybenzyl alcohol,CHEBI_17069,OCc1cccc(O)c1,"[3-methylphenol, O2, reduced [NADPH--hemoprote..."
2,A0A087X1C5,Putative cytochrome P450 2D7,an organic molecule + O2 + reduced [NADPH--hem...,17149,an organic molecule,CHEBI_142491,*[H],reduced [NADPH--hemoprotein reductase],heme,an alcohol,CHEBI_30879,O[*],"[an organic molecule, O2, reduced [NADPH--hemo..."
3,A0A0C5Q4Y6,Ferruginol synthase 1,"abieta-8,11,13-triene + O2 + reduced [NADPH--h...",48080,"abieta-8,11,13-triene",CHEBI_86062,CC(C)c1ccc2c(CC[C@H]3C(C)(C)CCC[C@]23C)c1,reduced [NADPH--hemoprotein reductase],heme,ferruginol,CHEBI_78274,CC(C)c1cc2CC[C@H]3C(C)(C)CCC[C@]3(C)c2cc1O,"[abieta-8,11,13-triene, O2, reduced [NADPH--he..."
4,A0A0C5Q4Y6,Ferruginol synthase 1,ferruginol + O2 + reduced [NADPH--hemoprotein ...,55428,ferruginol,CHEBI_78274,CC(C)c1cc2CC[C@H]3C(C)(C)CCC[C@]3(C)c2cc1O,reduced [NADPH--hemoprotein reductase],heme,11-hydroxyferruginol,CHEBI_138942,C=12[C@]3(C)[C@@]([H])(CCC1C=C(C(=C2O)O)C(C)C)...,"[ferruginol, O2, reduced [NADPH--hemoprotein r..."
5,A0A0C5Q4Y6,Ferruginol synthase 1,miltiradiene + 2 O2 + 2 reduced [NADPH--hemopr...,66796,miltiradiene,CHEBI_65037,[H][C@@]12CCC3=C(CC=C(C3)C(C)C)[C@@]1(C)CCCC2(C)C,2 reduced [NADPH--hemoprotein reductase],heme,11-oxomiltiradiene,CHEBI_167496,[H][C@@]12CCC3=C(C(=O)C=C(C3)C(C)C)[C@@]1(C)CC...,"[miltiradiene, 2 O2, 2 reduced [NADPH--hemopro..."
6,A0A0C5QRZ2,Ferruginol synthase,"abieta-8,11,13-triene + O2 + reduced [NADPH--h...",48080,"abieta-8,11,13-triene",CHEBI_86062,CC(C)c1ccc2c(CC[C@H]3C(C)(C)CCC[C@]23C)c1,reduced [NADPH--hemoprotein reductase],heme,ferruginol,CHEBI_78274,CC(C)c1cc2CC[C@H]3C(C)(C)CCC[C@]3(C)c2cc1O,"[abieta-8,11,13-triene, O2, reduced [NADPH--he..."
7,A0A0C5QRZ2,Ferruginol synthase,ferruginol + O2 + reduced [NADPH--hemoprotein ...,55428,ferruginol,CHEBI_78274,CC(C)c1cc2CC[C@H]3C(C)(C)CCC[C@]3(C)c2cc1O,reduced [NADPH--hemoprotein reductase],heme,11-hydroxyferruginol,CHEBI_138942,C=12[C@]3(C)[C@@]([H])(CCC1C=C(C(=C2O)O)C(C)C)...,"[ferruginol, O2, reduced [NADPH--hemoprotein r..."
8,A0A0C5QRZ2,Ferruginol synthase,miltiradiene + 2 O2 + 2 reduced [NADPH--hemopr...,66796,miltiradiene,CHEBI_65037,[H][C@@]12CCC3=C(CC=C(C3)C(C)C)[C@@]1(C)CCCC2(C)C,2 reduced [NADPH--hemoprotein reductase],heme,11-oxomiltiradiene,CHEBI_167496,[H][C@@]12CCC3=C(C(=O)C=C(C3)C(C)C)[C@@]1(C)CC...,"[miltiradiene, 2 O2, 2 reduced [NADPH--hemopro..."
9,A0A1D6HSP4,Dimethylnonatriene synthase,"(6E,10E)-geranyllinalool + O2 + reduced [NADPH...",13545,"(6E,10E)-geranyllinalool",CHEBI_74299,CC(C)=CCC\C(C)=C\CC\C(C)=C\CCC(C)(O)C=C,reduced [NADPH--hemoprotein reductase],heme,"(3E,7E)-4,8,12-trimethyltrideca 1,3,7,11-tetra...",CHEBI_74322;CHEBI_48058,CC(C)=CCC\C(C)=C\CC\C(C)=C\C=C;CC(=O)C=C,"[(6E,10E)-geranyllinalool, O2, reduced [NADPH-..."


I have a dataframe containing Uniprot ID,name, reaction, RHEA ID, substrates,
Substrate CHEBI ID, Substrate SMILES, reducing agent, cofactor, products, Product CHEBI ID, Product SMILES & the list of participants which I don't need anymore (but I'm still going to keep it for now).

Next things I want to do: add species & kingdom from taxa.csv, add sequences, lenghts (I have these in my google sheets db).

I'm missing chemical formula of product here - decide on how to add it to the database

In [22]:
taxa = pd.read_csv("./taxaNAT.csv",index_col='protein',sep=',')
taxa = taxa.fillna("MISSING VALUE")

taxa.head(20)

,organism_name,kingdom_name,superkingdom_name
protein,,,
A0A1H9YG21,Nitrosospira multiformis,MISSING VALUE,Bacteria
A0A1H8EWZ4,Nitrosospira multiformis,MISSING VALUE,Bacteria
A0A1I7H525,Nitrosospira multiformis,MISSING VALUE,Bacteria
A0A2T5IGS8,Nitrosospira multiformis,MISSING VALUE,Bacteria
Q577H8,Brucella abortus biovar 1 (strain 9-941),MISSING VALUE,Bacteria
A0A4X2K8V6,Vombatus ursinus,Metazoa,Eukaryota
A0A4X2MFM9,Vombatus ursinus,Metazoa,Eukaryota
A0A4X2KZM1,Vombatus ursinus,Metazoa,Eukaryota
A0A4X2KZT1,Vombatus ursinus,Metazoa,Eukaryota


In [23]:
taxa_sp_king = taxa.drop(columns=['superkingdom_name'])

In [24]:
merged_df = pd.merge(df, taxa, left_on='protein', right_index=True)
merged_df.head(20)


,protein,name,reaction,rheaID,substrates,Substrate CHEBI ID,Substrate SMILES,reducing_agent,cofactor,products,Product CHEBI ID,Product SMILES,participants,organism_name,kingdom_name,superkingdom_name
0,A0A075TMP8,Cytochrome P450 monooxygenase patI,3-hydroxybenzyl alcohol + O2 + reduced [NADPH-...,62212,3-hydroxybenzyl alcohol,CHEBI_17069,OCc1cccc(O)c1,reduced [NADPH--hemoprotein reductase],heme,gentisyl alcohol,CHEBI_5325,C=1(C=C(C(=CC1)O)CO)O,"[3-hydroxybenzyl alcohol, O2, reduced [NADPH--...",Penicillium expansum,Fungi,Eukaryota
1,A0A075TRL5,Cytochrome P450 monooxygenase patH,3-methylphenol + O2 + reduced [NADPH--hemoprot...,62208,3-methylphenol,CHEBI_17231,Cc1cccc(O)c1,reduced [NADPH--hemoprotein reductase],heme,3-hydroxybenzyl alcohol,CHEBI_17069,OCc1cccc(O)c1,"[3-methylphenol, O2, reduced [NADPH--hemoprote...",Penicillium expansum,Fungi,Eukaryota
2,A0A087X1C5,Putative cytochrome P450 2D7,an organic molecule + O2 + reduced [NADPH--hem...,17149,an organic molecule,CHEBI_142491,*[H],reduced [NADPH--hemoprotein reductase],heme,an alcohol,CHEBI_30879,O[*],"[an organic molecule, O2, reduced [NADPH--hemo...",Homo sapiens,Metazoa,Eukaryota
3,A0A0C5Q4Y6,Ferruginol synthase 1,"abieta-8,11,13-triene + O2 + reduced [NADPH--h...",48080,"abieta-8,11,13-triene",CHEBI_86062,CC(C)c1ccc2c(CC[C@H]3C(C)(C)CCC[C@]23C)c1,reduced [NADPH--hemoprotein reductase],heme,ferruginol,CHEBI_78274,CC(C)c1cc2CC[C@H]3C(C)(C)CCC[C@]3(C)c2cc1O,"[abieta-8,11,13-triene, O2, reduced [NADPH--he...",Rosmarinus officinalis,Viridiplantae,Eukaryota
4,A0A0C5Q4Y6,Ferruginol synthase 1,ferruginol + O2 + reduced [NADPH--hemoprotein ...,55428,ferruginol,CHEBI_78274,CC(C)c1cc2CC[C@H]3C(C)(C)CCC[C@]3(C)c2cc1O,reduced [NADPH--hemoprotein reductase],heme,11-hydroxyferruginol,CHEBI_138942,C=12[C@]3(C)[C@@]([H])(CCC1C=C(C(=C2O)O)C(C)C)...,"[ferruginol, O2, reduced [NADPH--hemoprotein r...",Rosmarinus officinalis,Viridiplantae,Eukaryota
5,A0A0C5Q4Y6,Ferruginol synthase 1,miltiradiene + 2 O2 + 2 reduced [NADPH--hemopr...,66796,miltiradiene,CHEBI_65037,[H][C@@]12CCC3=C(CC=C(C3)C(C)C)[C@@]1(C)CCCC2(C)C,2 reduced [NADPH--hemoprotein reductase],heme,11-oxomiltiradiene,CHEBI_167496,[H][C@@]12CCC3=C(C(=O)C=C(C3)C(C)C)[C@@]1(C)CC...,"[miltiradiene, 2 O2, 2 reduced [NADPH--hemopro...",Rosmarinus officinalis,Viridiplantae,Eukaryota
6,A0A0C5QRZ2,Ferruginol synthase,"abieta-8,11,13-triene + O2 + reduced [NADPH--h...",48080,"abieta-8,11,13-triene",CHEBI_86062,CC(C)c1ccc2c(CC[C@H]3C(C)(C)CCC[C@]23C)c1,reduced [NADPH--hemoprotein reductase],heme,ferruginol,CHEBI_78274,CC(C)c1cc2CC[C@H]3C(C)(C)CCC[C@]3(C)c2cc1O,"[abieta-8,11,13-triene, O2, reduced [NADPH--he...",Salvia fruticosa,Viridiplantae,Eukaryota
7,A0A0C5QRZ2,Ferruginol synthase,ferruginol + O2 + reduced [NADPH--hemoprotein ...,55428,ferruginol,CHEBI_78274,CC(C)c1cc2CC[C@H]3C(C)(C)CCC[C@]3(C)c2cc1O,reduced [NADPH--hemoprotein reductase],heme,11-hydroxyferruginol,CHEBI_138942,C=12[C@]3(C)[C@@]([H])(CCC1C=C(C(=C2O)O)C(C)C)...,"[ferruginol, O2, reduced [NADPH--hemoprotein r...",Salvia fruticosa,Viridiplantae,Eukaryota
8,A0A0C5QRZ2,Ferruginol synthase,miltiradiene + 2 O2 + 2 reduced [NADPH--hemopr...,66796,miltiradiene,CHEBI_65037,[H][C@@]12CCC3=C(CC=C(C3)C(C)C)[C@@]1(C)CCCC2(C)C,2 reduced [NADPH--hemoprotein reductase],heme,11-oxomiltiradiene,CHEBI_167496,[H][C@@]12CCC3=C(C(=O)C=C(C3)C(C)C)[C@@]1(C)CC...,"[miltiradiene, 2 O2, 2 reduced [NADPH--hemopro...",Salvia fruticosa,Viridiplantae,Eukaryota
9,A0A1D6HSP4,Dimethylnonatriene synthase,"(6E,10E)-geranyllinalool + O2 + reduced [NADPH...",13545,"(6E,10E)-geranyllinalool",CHEBI_74299,CC(C)=CCC\C(C)=C\CC\C(C)=C\CCC(C)(O)C=C,reduced [NADPH--hemoprotein reductase],heme,"(3E,7E)-4,8,12-trimethyltrideca 1,3,7,11-tetra...",CHEBI_74322;CHEBI_48058,CC(C)=CCC\C(C)=C\CC\C(C)=C\C=C;CC(=O)C=C,"[(6E,10E)-geranyllinalool, O2, reduced [NADPH-...",Zea mays,Viridiplantae,Eukaryota


In [25]:
merged_df.columns

Index(['protein', 'name', 'reaction', 'rheaID', 'substrates',
       'Substrate CHEBI ID', 'Substrate SMILES', 'reducing_agent', 'cofactor',
       'products', 'Product CHEBI ID', 'Product SMILES', 'participants',
       'organism_name', 'kingdom_name', 'superkingdom_name'],
      dtype='object')

In [26]:
kingdom_counts = merged_df.groupby('kingdom_name').count().sort_values('protein', ascending = False)

In [27]:
kingdom_counts.head(30)

,protein,name,reaction,rheaID,substrates,Substrate CHEBI ID,Substrate SMILES,reducing_agent,cofactor,products,Product CHEBI ID,Product SMILES,participants,organism_name,superkingdom_name
kingdom_name,,,,,,,,,,,,,,,
Metazoa,2239,2239,2117,2117,2117,2117,2117,2117,2117,2117,2117,2117,2117,2239,2239
Viridiplantae,705,705,505,505,505,505,505,505,505,505,505,505,505,705,705
Fungi,586,586,177,177,177,177,177,177,177,177,177,177,177,586,586
MISSING VALUE,248,248,128,128,128,125,125,128,128,128,125,125,128,248,248
Bamfordvirae,2,2,1,1,1,1,1,1,1,1,1,1,1,2,2


Metazoa = Animalia</br>
Viridiplantae = "green plantae" = Plantae</br>
Bamfordvirae = viruses</br>
Bacteria are in MISSING VALUE</br>


In [28]:
merged_df['kingdom_name'] = merged_df['kingdom_name'].str.replace("Metazoa","Animalia")
merged_df['kingdom_name'] = merged_df['kingdom_name'].str.replace("Viridiplantae","Plantae")
mask = (merged_df['kingdom_name'] == 'MISSING VALUE') & (merged_df['superkingdom_name'] == 'Bacteria')
merged_df.loc[mask, 'kingdom_name'] = 'Bacteria'

In [29]:
pfams = pd.read_csv("./pfams.csv", index_col='protein', sep=',')
pfams = pfams.fillna("MISSING VALUE")

pfams.head(20)

,pfam
protein,
A0A5N4AF60,PF00009
A0A3M7M744,PF00009
A0A8J5EMQ6,PF00009
A0A8H7BGB7,PF00009
A0A8H7PNS5,PF00009
A0A364NAN5,PF00009
A0A3M0WC62,PF00009
A0A7R9BNE8,PF00009
A0A3M7MT89,PF00009


In [30]:
merged_df_with_pfams = pd.merge(merged_df, pfams, left_on='protein', right_index=True)
merged_df_with_pfams.head(20)


,protein,name,reaction,rheaID,substrates,Substrate CHEBI ID,Substrate SMILES,reducing_agent,cofactor,products,Product CHEBI ID,Product SMILES,participants,organism_name,kingdom_name,superkingdom_name,pfam
0,A0A075TMP8,Cytochrome P450 monooxygenase patI,3-hydroxybenzyl alcohol + O2 + reduced [NADPH-...,62212,3-hydroxybenzyl alcohol,CHEBI_17069,OCc1cccc(O)c1,reduced [NADPH--hemoprotein reductase],heme,gentisyl alcohol,CHEBI_5325,C=1(C=C(C(=CC1)O)CO)O,"[3-hydroxybenzyl alcohol, O2, reduced [NADPH--...",Penicillium expansum,Fungi,Eukaryota,PF00067
1,A0A075TRL5,Cytochrome P450 monooxygenase patH,3-methylphenol + O2 + reduced [NADPH--hemoprot...,62208,3-methylphenol,CHEBI_17231,Cc1cccc(O)c1,reduced [NADPH--hemoprotein reductase],heme,3-hydroxybenzyl alcohol,CHEBI_17069,OCc1cccc(O)c1,"[3-methylphenol, O2, reduced [NADPH--hemoprote...",Penicillium expansum,Fungi,Eukaryota,PF00067
2,A0A087X1C5,Putative cytochrome P450 2D7,an organic molecule + O2 + reduced [NADPH--hem...,17149,an organic molecule,CHEBI_142491,*[H],reduced [NADPH--hemoprotein reductase],heme,an alcohol,CHEBI_30879,O[*],"[an organic molecule, O2, reduced [NADPH--hemo...",Homo sapiens,Animalia,Eukaryota,PF00067
3,A0A0C5Q4Y6,Ferruginol synthase 1,"abieta-8,11,13-triene + O2 + reduced [NADPH--h...",48080,"abieta-8,11,13-triene",CHEBI_86062,CC(C)c1ccc2c(CC[C@H]3C(C)(C)CCC[C@]23C)c1,reduced [NADPH--hemoprotein reductase],heme,ferruginol,CHEBI_78274,CC(C)c1cc2CC[C@H]3C(C)(C)CCC[C@]3(C)c2cc1O,"[abieta-8,11,13-triene, O2, reduced [NADPH--he...",Rosmarinus officinalis,Plantae,Eukaryota,PF00067
4,A0A0C5Q4Y6,Ferruginol synthase 1,ferruginol + O2 + reduced [NADPH--hemoprotein ...,55428,ferruginol,CHEBI_78274,CC(C)c1cc2CC[C@H]3C(C)(C)CCC[C@]3(C)c2cc1O,reduced [NADPH--hemoprotein reductase],heme,11-hydroxyferruginol,CHEBI_138942,C=12[C@]3(C)[C@@]([H])(CCC1C=C(C(=C2O)O)C(C)C)...,"[ferruginol, O2, reduced [NADPH--hemoprotein r...",Rosmarinus officinalis,Plantae,Eukaryota,PF00067
5,A0A0C5Q4Y6,Ferruginol synthase 1,miltiradiene + 2 O2 + 2 reduced [NADPH--hemopr...,66796,miltiradiene,CHEBI_65037,[H][C@@]12CCC3=C(CC=C(C3)C(C)C)[C@@]1(C)CCCC2(C)C,2 reduced [NADPH--hemoprotein reductase],heme,11-oxomiltiradiene,CHEBI_167496,[H][C@@]12CCC3=C(C(=O)C=C(C3)C(C)C)[C@@]1(C)CC...,"[miltiradiene, 2 O2, 2 reduced [NADPH--hemopro...",Rosmarinus officinalis,Plantae,Eukaryota,PF00067
6,A0A0C5QRZ2,Ferruginol synthase,"abieta-8,11,13-triene + O2 + reduced [NADPH--h...",48080,"abieta-8,11,13-triene",CHEBI_86062,CC(C)c1ccc2c(CC[C@H]3C(C)(C)CCC[C@]23C)c1,reduced [NADPH--hemoprotein reductase],heme,ferruginol,CHEBI_78274,CC(C)c1cc2CC[C@H]3C(C)(C)CCC[C@]3(C)c2cc1O,"[abieta-8,11,13-triene, O2, reduced [NADPH--he...",Salvia fruticosa,Plantae,Eukaryota,PF00067
7,A0A0C5QRZ2,Ferruginol synthase,ferruginol + O2 + reduced [NADPH--hemoprotein ...,55428,ferruginol,CHEBI_78274,CC(C)c1cc2CC[C@H]3C(C)(C)CCC[C@]3(C)c2cc1O,reduced [NADPH--hemoprotein reductase],heme,11-hydroxyferruginol,CHEBI_138942,C=12[C@]3(C)[C@@]([H])(CCC1C=C(C(=C2O)O)C(C)C)...,"[ferruginol, O2, reduced [NADPH--hemoprotein r...",Salvia fruticosa,Plantae,Eukaryota,PF00067
8,A0A0C5QRZ2,Ferruginol synthase,miltiradiene + 2 O2 + 2 reduced [NADPH--hemopr...,66796,miltiradiene,CHEBI_65037,[H][C@@]12CCC3=C(CC=C(C3)C(C)C)[C@@]1(C)CCCC2(C)C,2 reduced [NADPH--hemoprotein reductase],heme,11-oxomiltiradiene,CHEBI_167496,[H][C@@]12CCC3=C(C(=O)C=C(C3)C(C)C)[C@@]1(C)CC...,"[miltiradiene, 2 O2, 2 reduced [NADPH--hemopro...",Salvia fruticosa,Plantae,Eukaryota,PF00067
9,A0A1D6HSP4,Dimethylnonatriene synthase,"(6E,10E)-geranyllinalool + O2 + reduced [NADPH...",13545,"(6E,10E)-geranyllinalool",CHEBI_74299,CC(C)=CCC\C(C)=C\CC\C(C)=C\CCC(C)(O)C=C,reduced [NADPH--hemoprotein reductase],heme,"(3E,7E)-4,8,12-trimethyltrideca 1,3,7,11-tetra...",CHEBI_74322;CHEBI_48058,CC(C)=CCC\C(C)=C\CC\C(C)=C\C=C;CC(=O)C=C,"[(6E,10E)-geranyllinalool, O2, reduced [NADPH-...",Zea mays,Plantae,Eukaryota,PF00067


In [31]:
seqs = pd.read_csv("./sequencesNAT.csv",index_col='protein',sep=',')
seqs = seqs.fillna("MISSING VALUE")

seqs.head(20)

,sequence
protein,
P05108,MLAKGLPPRSVLVKGCQTFLSAPREGLGRLRVPTGEGAGISTRSPR...
Q9V7G5,MFVEKVLERTTNLELCSILILLVISLSIYTFYATLNTYLRSVLLSL...
O65815,MEIPYLLTTTLLLLFTTLYLLLRRRSSTLPPTIFPSLPIIGHLYLL...
Q9FMA5,MGAMMVMMGLLLIIVSLCSALLRWNQMRYTKNGLPPGTMGWPIFGE...
P08686,MLLLGLLLLPLLAGARLLWNWWKLRSLHLPPLAPGFLHLLQPDLPI...
P24557,MEALGFLKLEVNGPMVTVALSVALLALLKWYSTSAFSRLEKLGLRH...
Q9T0K2,MEMILITLCLTTLLALLLKSILKRTATKNFNLPPSPWRLPVIGNLH...
Q9LXM3,MSIFLCFLLLLPLSLILFKKLLPSKGKLPPGPIGLPIIGNLHQLGK...
Q9CX98,MSSLGDQRPAAGEQPGARLHVRATGGALLLCLLAVLLGWVWLRRQR...


In [32]:
merged_df_with_seqs = pd.merge(merged_df_with_pfams, seqs, left_on='protein', right_index=True)
merged_df_with_seqs.head(20)


,protein,name,reaction,rheaID,substrates,Substrate CHEBI ID,Substrate SMILES,reducing_agent,cofactor,products,Product CHEBI ID,Product SMILES,participants,organism_name,kingdom_name,superkingdom_name,pfam,sequence
0,A0A075TMP8,Cytochrome P450 monooxygenase patI,3-hydroxybenzyl alcohol + O2 + reduced [NADPH-...,62212,3-hydroxybenzyl alcohol,CHEBI_17069,OCc1cccc(O)c1,reduced [NADPH--hemoprotein reductase],heme,gentisyl alcohol,CHEBI_5325,C=1(C=C(C(=CC1)O)CO)O,"[3-hydroxybenzyl alcohol, O2, reduced [NADPH--...",Penicillium expansum,Fungi,Eukaryota,PF00067,MDILQLAPTHLLAILLSSTSALFLITYLLRAGHRPSDLPNGPPTVP...
1,A0A075TRL5,Cytochrome P450 monooxygenase patH,3-methylphenol + O2 + reduced [NADPH--hemoprot...,62208,3-methylphenol,CHEBI_17231,Cc1cccc(O)c1,reduced [NADPH--hemoprotein reductase],heme,3-hydroxybenzyl alcohol,CHEBI_17069,OCc1cccc(O)c1,"[3-methylphenol, O2, reduced [NADPH--hemoprote...",Penicillium expansum,Fungi,Eukaryota,PF00067,MEPFLLLLLVLLPAIVLVRYAFTYGHRTSTMPIGPPTLPFIGNIHQ...
2,A0A087X1C5,Putative cytochrome P450 2D7,an organic molecule + O2 + reduced [NADPH--hem...,17149,an organic molecule,CHEBI_142491,*[H],reduced [NADPH--hemoprotein reductase],heme,an alcohol,CHEBI_30879,O[*],"[an organic molecule, O2, reduced [NADPH--hemo...",Homo sapiens,Animalia,Eukaryota,PF00067,MGLEALVPLAMIVAIFLLLVDLMHRHQRWAARYPPGPLPLPGLGNL...
3,A0A0C5Q4Y6,Ferruginol synthase 1,"abieta-8,11,13-triene + O2 + reduced [NADPH--h...",48080,"abieta-8,11,13-triene",CHEBI_86062,CC(C)c1ccc2c(CC[C@H]3C(C)(C)CCC[C@]23C)c1,reduced [NADPH--hemoprotein reductase],heme,ferruginol,CHEBI_78274,CC(C)c1cc2CC[C@H]3C(C)(C)CCC[C@]3(C)c2cc1O,"[abieta-8,11,13-triene, O2, reduced [NADPH--he...",Rosmarinus officinalis,Plantae,Eukaryota,PF00067,MDSFPLLAALFFILAATWFISFRRPRNLPPGPFPYPIVGNMLQLGT...
4,A0A0C5Q4Y6,Ferruginol synthase 1,ferruginol + O2 + reduced [NADPH--hemoprotein ...,55428,ferruginol,CHEBI_78274,CC(C)c1cc2CC[C@H]3C(C)(C)CCC[C@]3(C)c2cc1O,reduced [NADPH--hemoprotein reductase],heme,11-hydroxyferruginol,CHEBI_138942,C=12[C@]3(C)[C@@]([H])(CCC1C=C(C(=C2O)O)C(C)C)...,"[ferruginol, O2, reduced [NADPH--hemoprotein r...",Rosmarinus officinalis,Plantae,Eukaryota,PF00067,MDSFPLLAALFFILAATWFISFRRPRNLPPGPFPYPIVGNMLQLGT...
5,A0A0C5Q4Y6,Ferruginol synthase 1,miltiradiene + 2 O2 + 2 reduced [NADPH--hemopr...,66796,miltiradiene,CHEBI_65037,[H][C@@]12CCC3=C(CC=C(C3)C(C)C)[C@@]1(C)CCCC2(C)C,2 reduced [NADPH--hemoprotein reductase],heme,11-oxomiltiradiene,CHEBI_167496,[H][C@@]12CCC3=C(C(=O)C=C(C3)C(C)C)[C@@]1(C)CC...,"[miltiradiene, 2 O2, 2 reduced [NADPH--hemopro...",Rosmarinus officinalis,Plantae,Eukaryota,PF00067,MDSFPLLAALFFILAATWFISFRRPRNLPPGPFPYPIVGNMLQLGT...
6,A0A0C5QRZ2,Ferruginol synthase,"abieta-8,11,13-triene + O2 + reduced [NADPH--h...",48080,"abieta-8,11,13-triene",CHEBI_86062,CC(C)c1ccc2c(CC[C@H]3C(C)(C)CCC[C@]23C)c1,reduced [NADPH--hemoprotein reductase],heme,ferruginol,CHEBI_78274,CC(C)c1cc2CC[C@H]3C(C)(C)CCC[C@]3(C)c2cc1O,"[abieta-8,11,13-triene, O2, reduced [NADPH--he...",Salvia fruticosa,Plantae,Eukaryota,PF00067,MDPFPLVAAALFIAATWFITFKRRRNLPPGPFPYPIVGNMLQLGSQ...
7,A0A0C5QRZ2,Ferruginol synthase,ferruginol + O2 + reduced [NADPH--hemoprotein ...,55428,ferruginol,CHEBI_78274,CC(C)c1cc2CC[C@H]3C(C)(C)CCC[C@]3(C)c2cc1O,reduced [NADPH--hemoprotein reductase],heme,11-hydroxyferruginol,CHEBI_138942,C=12[C@]3(C)[C@@]([H])(CCC1C=C(C(=C2O)O)C(C)C)...,"[ferruginol, O2, reduced [NADPH--hemoprotein r...",Salvia fruticosa,Plantae,Eukaryota,PF00067,MDPFPLVAAALFIAATWFITFKRRRNLPPGPFPYPIVGNMLQLGSQ...
8,A0A0C5QRZ2,Ferruginol synthase,miltiradiene + 2 O2 + 2 reduced [NADPH--hemopr...,66796,miltiradiene,CHEBI_65037,[H][C@@]12CCC3=C(CC=C(C3)C(C)C)[C@@]1(C)CCCC2(C)C,2 reduced [NADPH--hemoprotein reductase],heme,11-oxomiltiradiene,CHEBI_167496,[H][C@@]12CCC3=C(C(=O)C=C(C3)C(C)C)[C@@]1(C)CC...,"[miltiradiene, 2 O2, 2 reduced [NADPH--hemopro...",Salvia fruticosa,Plantae,Eukaryota,PF00067,MDPFPLVAAALFIAATWFITFKRRRNLPPGPFPYPIVGNMLQLGSQ...
9,A0A1D6HSP4,Dimethylnonatriene synthase,"(6E,10E)-ger

In [33]:
merged_df_with_seqs['length'] = merged_df_with_seqs['sequence'].apply(len)
merged_df_with_seqs.head(20)

,protein,name,reaction,rheaID,substrates,Substrate CHEBI ID,Substrate SMILES,reducing_agent,cofactor,products,Product CHEBI ID,Product SMILES,participants,organism_name,kingdom_name,superkingdom_name,pfam,sequence,length
0,A0A075TMP8,Cytochrome P450 monooxygenase patI,3-hydroxybenzyl alcohol + O2 + reduced [NADPH-...,62212,3-hydroxybenzyl alcohol,CHEBI_17069,OCc1cccc(O)c1,reduced [NADPH--hemoprotein reductase],heme,gentisyl alcohol,CHEBI_5325,C=1(C=C(C(=CC1)O)CO)O,"[3-hydroxybenzyl alcohol, O2, reduced [NADPH--...",Penicillium expansum,Fungi,Eukaryota,PF00067,MDILQLAPTHLLAILLSSTSALFLITYLLRAGHRPSDLPNGPPTVP...,526
1,A0A075TRL5,Cytochrome P450 monooxygenase patH,3-methylphenol + O2 + reduced [NADPH--hemoprot...,62208,3-methylphenol,CHEBI_17231,Cc1cccc(O)c1,reduced [NADPH--hemoprotein reductase],heme,3-hydroxybenzyl alcohol,CHEBI_17069,OCc1cccc(O)c1,"[3-methylphenol, O2, reduced [NADPH--hemoprote...",Penicillium expansum,Fungi,Eukaryota,PF00067,MEPFLLLLLVLLPAIVLVRYAFTYGHRTSTMPIGPPTLPFIGNIHQ...,524
2,A0A087X1C5,Putative cytochrome P450 2D7,an organic molecule + O2 + reduced [NADPH--hem...,17149,an organic molecule,CHEBI_142491,*[H],reduced [NADPH--hemoprotein reductase],heme,an alcohol,CHEBI_30879,O[*],"[an organic molecule, O2, reduced [NADPH--hemo...",Homo sapiens,Animalia,Eukaryota,PF00067,MGLEALVPLAMIVAIFLLLVDLMHRHQRWAARYPPGPLPLPGLGNL...,515
3,A0A0C5Q4Y6,Ferruginol synthase 1,"abieta-8,11,13-triene + O2 + reduced [NADPH--h...",48080,"abieta-8,11,13-triene",CHEBI_86062,CC(C)c1ccc2c(CC[C@H]3C(C)(C)CCC[C@]23C)c1,reduced [NADPH--hemoprotein reductase],heme,ferruginol,CHEBI_78274,CC(C)c1cc2CC[C@H]3C(C)(C)CCC[C@]3(C)c2cc1O,"[abieta-8,11,13-triene, O2, reduced [NADPH--he...",Rosmarinus officinalis,Plantae,Eukaryota,PF00067,MDSFPLLAALFFILAATWFISFRRPRNLPPGPFPYPIVGNMLQLGT...,493
4,A0A0C5Q4Y6,Ferruginol synthase 1,ferruginol + O2 + reduced [NADPH--hemoprotein ...,55428,ferruginol,CHEBI_78274,CC(C)c1cc2CC[C@H]3C(C)(C)CCC[C@]3(C)c2cc1O,reduced [NADPH--hemoprotein reductase],heme,11-hydroxyferruginol,CHEBI_138942,C=12[C@]3(C)[C@@]([H])(CCC1C=C(C(=C2O)O)C(C)C)...,"[ferruginol, O2, reduced [NADPH--hemoprotein r...",Rosmarinus officinalis,Plantae,Eukaryota,PF00067,MDSFPLLAALFFILAATWFISFRRPRNLPPGPFPYPIVGNMLQLGT...,493
5,A0A0C5Q4Y6,Ferruginol synthase 1,miltiradiene + 2 O2 + 2 reduced [NADPH--hemopr...,66796,miltiradiene,CHEBI_65037,[H][C@@]12CCC3=C(CC=C(C3)C(C)C)[C@@]1(C)CCCC2(C)C,2 reduced [NADPH--hemoprotein reductase],heme,11-oxomiltiradiene,CHEBI_167496,[H][C@@]12CCC3=C(C(=O)C=C(C3)C(C)C)[C@@]1(C)CC...,"[miltiradiene, 2 O2, 2 reduced [NADPH--hemopro...",Rosmarinus officinalis,Plantae,Eukaryota,PF00067,MDSFPLLAALFFILAATWFISFRRPRNLPPGPFPYPIVGNMLQLGT...,493
6,A0A0C5QRZ2,Ferruginol synthase,"abieta-8,11,13-triene + O2 + reduced [NADPH--h...",48080,"abieta-8,11,13-triene",CHEBI_86062,CC(C)c1ccc2c(CC[C@H]3C(C)(C)CCC[C@]23C)c1,reduced [NADPH--hemoprotein reductase],heme,ferruginol,CHEBI_78274,CC(C)c1cc2CC[C@H]3C(C)(C)CCC[C@]3(C)c2cc1O,"[abieta-8,11,13-triene, O2, reduced [NADPH--he...",Salvia fruticosa,Plantae,Eukaryota,PF00067,MDPFPLVAAALFIAATWFITFKRRRNLPPGPFPYPIVGNMLQLGSQ...,492
7,A0A0C5QRZ2,Ferruginol synthase,ferruginol + O2 + reduced [NADPH--hemoprotein ...,55428,ferruginol,CHEBI_78274,CC(C)c1cc2CC[C@H]3C(C)(C)CCC[C@]3(C)c2cc1O,reduced [NADPH--hemoprotein reductase],heme,11-hydroxyferruginol,CHEBI_138942,C=12[C@]3(C)[C@@]([H])(CCC1C=C(C(=C2O)O)C(C)C)...,"[ferruginol, O2, reduced [NADPH--hemoprotein r...",Salvia fruticosa,Plantae,Eukaryota,PF00067,MDPFPLVAAALFIAATWFITFKRRRNLPPGPFPYPIVGNMLQLGSQ...,492
8,A0A0C5QRZ2,Ferruginol synthase,miltiradiene + 2 O2 + 2 reduced [NADPH--hemopr...,66796,miltiradiene,CHEBI_65037,[H][C@@]12CCC3=C(CC=C(C3)C(C)C)[C@@]1(C)CCCC2(C)C,2 reduced [NADPH--hemoprotein reductase],heme,11-oxomiltiradiene,CHEBI_167496,[H][C@@]12CCC3=C(C(=O)C=C(C3)C(C)C)[C@@]1(C)CC...,"[miltiradiene, 2 O2, 2 reduced [NADPH--hemopro...",Salvia fruticosa,Plantae,Eukaryota,PF00067,MDPFPLVAAALFIAATWFITFKRRRNLPPGPFPYPIVGNMLQLGSQ...,492
9,A0A1D6HSP

In [34]:
#getting rid of 'CHEBI_:' in my df to simplify the foundation of CHEBI IDs and SMILES in reaction_data.csv file
merged_df_with_seqs['Substrate CHEBI ID'] = merged_df_with_seqs['Substrate CHEBI ID'].str.replace("CHEBI_","")
merged_df_with_seqs['Product CHEBI ID'] = merged_df_with_seqs['Product CHEBI ID'].str.replace("CHEBI_","")



In [35]:
merged_df_with_seqs.drop(columns='participants', inplace=True)
merged_df_with_seqs.to_csv('reactionsfromjson.csv', index=False)